# Caltech Machine Learning Homework # 6

In [1]:
import numpy as np
from sklearn.linear_model import Perceptron
import random
import math
import matplotlib.pyplot as plt
from typing import List
from itertools import product
import scipy.special
from scipy import optimize
import scipy.optimize as spo
from sympy import Symbol, Derivative

def dbg():
    import pdb; pdb.set_trace()

Instructions: https://work.caltech.edu/homework/hw6.pdf

Answers: http://work.caltech.edu/homework/hw6_sol.pdf

## Overfitting and Deterministic Noise

![](imgs/overfitting.png)

**[b]**